<a href="https://colab.research.google.com/github/MonicaChT16/RNN_sentimientosPelicula/blob/main/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install tensorflow

#CARGAR LIBRERIAS A UTILIZAR

In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
splits = {'train': 'data/train-00000-of-00001-40d9940488cb84b1.parquet', 'test': 'data/test-00000-of-00001-9cc7bacfd2a73ac7.parquet'}
datos = pd.read_parquet("hf://datasets/alexcom/analisis-sentimientos-textos-turisitcos-mx-polaridad/" + splits["train"])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


# **REVISAR LOS DATOS**

In [4]:
datos.head()

,text,label
0,el mejor lugar para comer sushi. excelente lug...,5
1,vista hermosa!. me sorprendió la maravillosa v...,5
2,"desastroso. buenas noches, ante todo explicar...",1
3,todo bien excepto que se niegan a dar vasos de...,3
4,el mejor zoologico de méxico. el ambiente es m...,5


In [5]:
datos.tail()

,text,label
176187,"gran hotel con buen servicio. hotel muy bueno,...",5
176188,estancia cómoda en la frontera de puebla y tla...,5
176189,encantador. realmente nos gustó quedarnos en e...,5
176190,en general la atención del personal fue excele...,5
176191,una bellesa en la ciudad. exelente lugar debaj...,5


In [6]:
datos.describe()

,label
count,176192.000000
mean,4.412839
std,0.925233
min,1.000000
25%,4.000000
50%,5.000000
75%,5.000000
max,5.000000


# **LIMPIEZA DE DATOS**

In [7]:
datos.isnull().sum()

,0
text,0
label,0


In [8]:
datos.duplicated().sum()

np.int64(3607)

In [ ]:
datos[datos.duplicated()]

,Descripcion,Puntuacion
1190,gran aporte de la universidad veracruzana. cue...,5
1806,ideal para iniciar el recorrido de museos. sin...,5
2136,ideal para iniciar el recorrido de museos. sin...,5
2231,socios. la hostess samantha muy amable atenta ...,5
2305,imperdible.. se trata de un lugar que encierra...,5
...,...,...
175757,visita. excelente servicio por parte del perso...,5
175915,socios. la hostess samantha muy amable atenta ...,5
175931,mala organización. el lugar es una maravilla q...,2
176019,visita a veracruz. es muy bueno ya que present...,5


In [9]:
datos.drop_duplicates(inplace=True)

In [10]:
datos.columns = ['Descripcion', 'Puntuacion']

In [ ]:
#CAMBIAR NOMBRE DE COLUMNA, AGREGAR ID

In [11]:
datos["Puntuacion"].value_counts()

,count
Puntuacion,
5,107493
4,41589
3,14924
2,4732
1,3847


#**BALANCEAR DATASET**

In [12]:
datos[datos["Puntuacion"]==5].value_counts()

,,count
Descripcion,Puntuacion,
🫶🏼. muy deliciosa la comida y una vista muy hermosa. una bonita experiencia lo recomiendo demasiado \n🫶🏼🫶🏼🫶🏼,5,1
"""el restaurante del país de las maravillas"". excelente todo, el lugar , el servicio , la comida , la música todo va a la altura!!! uno de los mejores restaurantes fusión que hacia falta en bogotá 👍👌",5,1
"excelente atencion y comodidad. el hotel mision es una excelente alternativa a la hora de visitar el sureste mexicano, ya que las instalaciones son muy representativas de la arquitectura mexicana, y esto hace que la estancia sea placentera y te haga sentir como en casa. la atencion del personal es excelente ya que te tratan como de tu familia. la comida en el restaurante es muy rica, ademas de todo que los precios tanto de hospedaje como del restaurante me parecen muy accesibles.",5,1
"me encanto. me gusto mucho porque la atención es excelente y tiene comida al gurmet y también a cualquier hora encuentras bocadillos, la playa esta muy limpia no se si la limpien pero esta hermosa y también me gusto el cuidado que tienen de reciclar y en especial lo de oir música todo el dia en el hotel y las albercas es lo mejor.....",5,1
"muy rico. puedes disfrutar de un buen corte de carne acompañado de una rica ensalada y sus respectivos condimentos, sin olvidar las quesadillas de queso menonita.",5,1
...,...,...
"!!!!dos pulgares arriba!!!!!. la palabra para describir el lugar es: excelencia. nuestro grupo estaba compuesto por 24 personas y verdaderamente todos salimos complacidos, el lugar tiene magia y un gran encanto. la comida es exquisita y los platos de entrantes gratamente sorprendentes. el trato de las chicas y...más",5,1
!!!!cumpleañossss!!!!!. este lugar es para mi uno de mis favoritos de la habana.\nyo soy amante de carne de res y la variedad de platos q ahí se presentan es espectacular respecto a este tipo de carne y en general.\nla cena estuvo especual. cene muy...más,5,1
!!!!!. qué lugar majestuoso!!! que impresiona cada persona que va allí!! lo recomiendo encarecidamente!!! viaje agradable con mis hijas!!!,5,1


In [16]:
datos['Puntuacion'].unique()

array([5, 1, 3, 4, 2])

In [17]:
clases=[]
for puntuacion in datos['Puntuacion'].unique():
  clase = datos[datos['Puntuacion'] == puntuacion]
  clase_balanceada = resample(clase,replace=True,n_samples=3847,random_state=42)
  clases.append(clase_balanceada)

In [23]:
data=pd.concat(clases).sample(frac=1,random_state=42).reset_index(drop=True)

In [24]:
data["Puntuacion"].value_counts()

,count
Puntuacion,
5,3847
3,3847
1,3847
2,3847
4,3847


In [37]:
#ONEHOTENCODING Puntuacion 20726
pd.concat([data.Puntuacion,pd.get_dummies(data.Puntuacion, dtype='int')], axis=1)

,Puntuacion,1,2,3,4,5
0,5,0,0,0,0,1
1,3,0,0,1,0,0
2,1,1,0,0,0,0
3,5,0,0,0,0,1
4,3,0,0,1,0,0
...,...,...,...,...,...,...
19230,3,0,0,1,0,0
19231,4,0,0,0,1,0
19232,1,1,0,0,0,0
19233,5,0,0,0,0,1


In [42]:
data_cuali=pd.get_dummies(data.Puntuacion, dtype='int')
data=data.drop('Puntuacion', axis=1)
data=pd.concat([data,data_cuali],axis=1)

In [43]:
data

,Descripcion,1,2,3,4,5
0,proximos pasajeros. hola ! el 17 de abril esta...,0,0,0,0,1
1,podría ser un lugar de democracia.. como mucho...,0,0,1,0,0
2,"desastroso. restaurante viejo, mal servicio, t...",1,0,0,0,0
3,parque de primer mundo. lo amamos. este es un ...,0,0,0,0,1
4,no olivdes nada en las habitaciones ya que no ...,0,0,1,0,0
...,...,...,...,...,...,...
19230,un poco caro para el viaje. no se puede supera...,0,0,1,0,0
19231,estancia confortable. nos alojamos durante un ...,0,0,0,1,0
19232,servicio impactante.. reservamos debido a la p...,1,0,0,0,0
19233,"almuerzo con amigas. en el restaurante ushin, ...",0,0,0,0,1


In [38]:
data

,Descripcion,Puntuacion
0,proximos pasajeros. hola ! el 17 de abril esta...,5
1,podría ser un lugar de democracia.. como mucho...,3
2,"desastroso. restaurante viejo, mal servicio, t...",1
3,parque de primer mundo. lo amamos. este es un ...,5
4,no olivdes nada en las habitaciones ya que no ...,3
...,...,...
19230,un poco caro para el viaje. no se puede supera...,3
19231,estancia confortable. nos alojamos durante un ...,4
19232,servicio impactante.. reservamos debido a la p...,1
19233,"almuerzo con amigas. en el restaurante ushin, ...",5


#**DIVIDIR EL DATASET**

In [62]:
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [63]:
print(train_data.shape)
print(test_data.shape)

(15388, 6)
(3847, 6)


# **PREPROCESAMIENTO DE LA DATA**

In [64]:
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(train_data["Descripcion"])
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["Descripcion"]), maxlen=300)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data["Descripcion"]), maxlen=300)

In [65]:
print(X_train)

[[   0    0    0 ...    3  218   78]
 [   0    0    0 ...   17   14  590]
 [   0    0    0 ...    1  308 8386]
 ...
 [   0    0    0 ...    3 3636 4193]
 [   0    0    0 ...   35   39  565]
 [   0    0    0 ...  133    2   29]]


In [81]:
print(X_test)

[[   0    0    0 ...    4    5  812]
 [   0    0    0 ...  184    1 4577]
 [   0    0    0 ...  163  644   26]
 ...
 [   0    0    0 ...  528   24  862]
 [   0    0    0 ...  254 1217   20]
 [   0    0    0 ...   10   75 1101]]


In [82]:
print(X_train.shape)

(15388, 300)


In [83]:
print(X_test.shape)

(3847, 300)


In [74]:
Y_train = train_data[[1, 2, 3, 4, 5]]
Y_test = test_data[[1, 2, 3, 4, 5]]

In [75]:
print(Y_train)

       1  2  3  4  5
9276   0  0  1  0  0
7808   0  0  1  0  0
16766  0  0  0  1  0
7285   0  0  0  0  1
15712  1  0  0  0  0
...   .. .. .. .. ..
11284  0  0  0  1  0
11964  1  0  0  0  0
5390   0  0  1  0  0
860    0  1  0  0  0
15795  1  0  0  0  0

[15388 rows x 5 columns]


# **LSTM**

In [76]:
model = Sequential()
model.add(Embedding(input_dim=10000, output_dim=128, input_length=300))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(5, activation="softmax"))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [77]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [85]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [84]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

# **ENTRENANDO MODELO**

In [86]:
model.fit(X_train, Y_train, epochs=5, batch_size=64, validation_split=0.2)

Epoch 1/5
193/193 ━━━━━━━━━━━━━━━━━━━━ 201s 1s/step - accuracy: 0.2956 - loss: 1.4918 - val_accuracy: 0.4366 - val_loss: 1.2217
Epoch 2/5
193/193 ━━━━━━━━━━━━━━━━━━━━ 205s 1s/step - accuracy: 0.5030 - loss: 1.1047 - val_accuracy: 0.5270 - val_loss: 1.0737
Epoch 3/5
193/193 ━━━━━━━━━━━━━━━━━━━━ 192s 973ms/step - accuracy: 0.6314 - loss: 0.8935 - val_accuracy: 0.5500 - val_loss: 1.0593
Epoch 4/5
193/193 ━━━━━━━━━━━━━━━━━━━━ 201s 968ms/step - accuracy: 0.7274 - loss: 0.6998 - val_accuracy: 0.5448 - val_loss: 1.1268
Epoch 5/5
193/193 ━━━━━━━━━━━━━━━━━━━━ 187s 968ms/step - accuracy: 0.7928 - loss: 0.5445 - val_accuracy: 0.5396 - val_loss: 1.2506


In [87]:
loss, accuracy = model.evaluate(X_test, Y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

121/121 ━━━━━━━━━━━━━━━━━━━━ 14s 118ms/step - accuracy: 0.5676 - loss: 1.1727
Test Loss: 1.2096716165542603
Test Accuracy: 0.5560176968574524


In [88]:
def predecir_sentimiento(texto):
  # tokenize and pad the review
  sequence = tokenizer.texts_to_sequences([texto])
  padded_sequence = pad_sequences(sequence, maxlen=300)
  prediction = model.predict(padded_sequence)
  # sentiment = "positive" if prediction[0][0] > 0.5 else "negative"  0.2
  predicted_class = prediction.argmax(axis=-1)[0]  # `argmax` devuelve el índice de la clase con mayor probabilidad

    # Mapear el índice de la clase a la puntuación
  class_mapping = {0: 1, 1: 2, 2: 3, 3: 4, 4: 5}  # Aquí mapeas el índice de la clase a la puntuación correspondiente
  sentiment = class_mapping[predicted_class]

  return sentiment

In [89]:
nueva_resena = "Esta película fue fantástica. Me encantó."
sentimiento = predecir_sentimiento(nueva_resena)
print(f"El sentimiento de la reseña es: {sentimiento}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 507ms/step
El sentimiento de la reseña es: 5


In [90]:
nueva_resena = "Horrible, no la recomendaría a nadie."
sentimiento = predecir_sentimiento(nueva_resena)
print(f"El sentimiento de la reseña es: {sentimiento}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step
El sentimiento de la reseña es: 1


In [ ]:
# nueva_resena = "Esta película fue fantástica. Me encantó."
# Sentimiento positivo, probablemente puntuación 5.

# nueva_resena = "El filme fue aceptable, pero no cumplió mis expectativas."
# Sentimiento neutro, probablemente puntuación 3.

# nueva_resena = "Horrible, no la recomendaría a nadie."
# Sentimiento negativo, probablemente puntuación 1.

# nueva_resena = "La historia es muy aburrida, pero las actuaciones son buenas."
# Sentimiento mixto, probablemente puntuación 2 o 3.

# nueva_resena = "Una de las mejores películas que he visto. Totalmente recomendada."
# Sentimiento extremadamente positivo, probablemente puntuación 5.

# nueva_resena = "El guion era predecible, pero tiene buenos efectos especiales."
# Sentimiento moderadamente negativo, probablemente puntuación 3 o 4.

# nueva_resena = "No pude ver toda la película. Me quedé dormido después de 30 minutos."
# Sentimiento negativo, probablemente puntuación 1 o 2.

# nueva_resena = "Excelente película para toda la familia. Me hizo reír mucho."
# Sentimiento positivo, probablemente puntuación 4 o 5.

# nueva_resena = "No entiendo por qué a la gente le gusta esta película. Me pareció mala."
# Sentimiento negativo, probablemente puntuación 1 o 2.

# nueva_resena = "La película tiene momentos interesantes, pero en general me resultó muy larga."
# Sentimiento mixto, probablemente puntuación 3.